In [196]:
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")
import re

In [197]:
df_old = pd.read_csv('CF18-07-2022-to-18-10-2022.csv')
df = pd.read_csv('CF-06-08-2022-to-06-11-2022.csv')
print(df_old.columns == df.columns)
df.head()

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]


,SYMBOL \n,COMPANY \n,REGULATION \n,NAME OF THE ACQUIRER/DISPOSER \n,CATEGORY OF PERSON \n,TYPE OF SECURITY (PRIOR) \n,NO. OF SECURITY (PRIOR) \n,% SHAREHOLDING (PRIOR) \n,TYPE OF SECURITY (ACQUIRED/DISPLOSED) \n,NO. OF SECURITIES (ACQUIRED/DISPLOSED) \n,...,DERIVATIVE TYPE SECURITY \n,DERIVATIVE CONTRACT SPECIFICATION \n,NOTIONAL VALUE(BUY) \n,NUMBER OF UNITS/CONTRACT LOT SIZE (BUY) \n,NOTIONAL VALUE(SELL) \n,NUMBER OF UNITS/CONTRACT LOT SIZE (SELL) \n,EXCHANGE \n,REMARK \n,BROADCASTE DATE AND TIME \n,XBRL \n
0,VCL,Vaxtex Cotfab Limited,7(2),Qmin Pharma Private Limited,Promoters,Equity Shares,15093246,30.07,NaN,75766.0,...,-,-,0,0,0,0,NSE,-,05-Nov-2022 16:57,https://archives.nseindia.com/corporate/xbrl/I...
1,BBL,Bharat Bijlee Limited,7(2),AVANTI PRAHLAD MEHTA,Promoter Group,Equity Shares,84320,1.49,NaN,11520.0,...,-,-,0,0,0,0,NaN,-,05-Nov-2022 12:36,https://archives.nseindia.com/corporate/xbrl/I...
2,ROUTE,ROUTE MOBILE LIMITED,7(2),Route Mobile Employee Welfare Trust,Other,Equity Shares,128975,0.21,NaN,90525.0,...,-,-,0,0,0,0,NaN,-,04-Nov-2022 23:19,https://archives.nseindia.com/corporate/xbrl/I...
3,ROUTE,ROUTE MOBILE LIMITED,7(2),Route Mobile Employee Welfare Trust,Other,Equity Shares,128975,0.21,NaN,88000.0,...,-,-,0,0,0,0,NaN,-,04-Nov-2022 23:19,https://archives.nseindia.com/corporate/xbrl/I...
4,AURIONPRO,Aurionpro Solutions Limited,7(2),ASHISH RAI,Director,Equity Shares,1798965,7.89,NaN,1000.0,...,-,-,0,0,0,0,NSE,-,04-Nov-2022 23:19,https://archives.nseindia.com/corporate/xbrl/I...


---
## Renaming All Columns Excluding Unnecessery Ones .

In [198]:
df = df[[
    'SYMBOL \n', 'CATEGORY OF PERSON \n',
    'NO. OF SECURITIES (ACQUIRED/DISPLOSED) \n',
    'VALUE OF SECURITY (ACQUIRED/DISPLOSED) \n', 'MODE OF ACQUISITION \n',
    'ACQUISITION/DISPOSAL TRANSACTION TYPE \n', 'TYPE OF SECURITY (POST) \n',
    '% SHAREHOLDING (PRIOR) \n', '% POST \n'
]]


df.rename(columns={'SYMBOL \n':'SYMBOL','CATEGORY OF PERSON \n':'CATEGORY_OF_PERSON','NO. OF SECURITIES (ACQUIRED/DISPLOSED) \n':'NO_OF_SECURITIES_ACQUIRED','VALUE OF SECURITY (ACQUIRED/DISPLOSED) \n':'VALUE_OF_SECURITY_ACQUIRED','ACQUISITION/DISPOSAL TRANSACTION TYPE \n':'ACQUISITION_TRANSACTION_TYPE','MODE OF ACQUISITION \n':'MODE_OF_ACQUISITION','% SHAREHOLDING (PRIOR) \n':'%_SHAREHOLDING_PRIOR','% POST \n':'%_SHAREHOLDING_POST','TYPE OF SECURITY (POST) \n':'TYPE_OF_SECURITY'},inplace=True)

In [199]:
df.head()

,SYMBOL,CATEGORY_OF_PERSON,NO_OF_SECURITIES_ACQUIRED,VALUE_OF_SECURITY_ACQUIRED,MODE_OF_ACQUISITION,ACQUISITION_TRANSACTION_TYPE,TYPE_OF_SECURITY,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST
0,VCL,Promoters,75766.0,876147,Market Sale,Sell,Equity Shares,30.07,29.92
1,BBL,Promoter Group,11520.0,0,Off Market,Buy,Equity Shares,1.49,1.7
2,ROUTE,Other,90525.0,119588051,-,Pledge,Equity Shares,0.21,0.21
3,ROUTE,Other,88000.0,117392000,-,Pledge Revoke,Equity Shares,0.21,0.21
4,AURIONPRO,Director,1000.0,370000,Market Purchase,Buy,Equity Shares,7.89,7.89


---
## Preparing Dataset For Further Analysis.

In [200]:
df = df[df['ACQUISITION_TRANSACTION_TYPE'] == 'Buy']
df = df[df['TYPE_OF_SECURITY'] == 'Equity Shares']
df = df[df['TYPE_OF_SECURITY'] == 'Equity Shares']
df = df[df['CATEGORY_OF_PERSON'].isin(['Promoters','Promoter Group'])]
df = df[df['MODE_OF_ACQUISITION'] == 'Market Purchase']


df = df[[
    'SYMBOL', 'NO_OF_SECURITIES_ACQUIRED', 'VALUE_OF_SECURITY_ACQUIRED',
    '%_SHAREHOLDING_PRIOR', '%_SHAREHOLDING_POST'
]]

In [201]:
a = df.copy()
df.head()

,SYMBOL,NO_OF_SECURITIES_ACQUIRED,VALUE_OF_SECURITY_ACQUIRED,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST
12,DBOL,12000.0,2039760,16.64,16.66
13,SPLPETRO,30000.0,24069369,0.42,0.45
14,SPLPETRO,10633.0,8455239,0.34,0.35
26,SHALBY,20000.0,2897682,7.18,7.2
32,MEP,328000.0,3922500,9.19,9.37


---
## Calculations Start Here.

### Finding Total Values And No Of Securities Bought.

In [202]:
df.head()

,SYMBOL,NO_OF_SECURITIES_ACQUIRED,VALUE_OF_SECURITY_ACQUIRED,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST
12,DBOL,12000.0,2039760,16.64,16.66
13,SPLPETRO,30000.0,24069369,0.42,0.45
14,SPLPETRO,10633.0,8455239,0.34,0.35
26,SHALBY,20000.0,2897682,7.18,7.2
32,MEP,328000.0,3922500,9.19,9.37


In [203]:
t_val = df[['SYMBOL','NO_OF_SECURITIES_ACQUIRED','VALUE_OF_SECURITY_ACQUIRED']]


In [204]:
t_val.head()
t_val = t_val.astype({'VALUE_OF_SECURITY_ACQUIRED':'float','NO_OF_SECURITIES_ACQUIRED':'float'})
t_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531 entries, 12 to 4852
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   SYMBOL                      531 non-null    object 
 1   NO_OF_SECURITIES_ACQUIRED   531 non-null    float64
 2   VALUE_OF_SECURITY_ACQUIRED  531 non-null    float64
dtypes: float64(2), object(1)
memory usage: 16.6+ KB


In [205]:
val = pd.DataFrame(t_val.groupby('SYMBOL')['VALUE_OF_SECURITY_ACQUIRED'].sum()).reset_index()
val.head()

,SYMBOL,VALUE_OF_SECURITY_ACQUIRED
0,ADANIENT,1.278702e+10
1,ADANIGREEN,9.351393e+09
2,ADANIPORTS,8.293400e+04
3,ADANITRANS,3.565280e+05
4,AIROLAM,7.691300e+04


In [206]:
t_sec = pd.DataFrame(t_val.groupby('SYMBOL')['NO_OF_SECURITIES_ACQUIRED'].sum()).reset_index()
t_sec.head()

,SYMBOL,NO_OF_SECURITIES_ACQUIRED
0,ADANIENT,3486140.0
1,ADANIGREEN,4000100.0
2,ADANIPORTS,100.0
3,ADANITRANS,100.0
4,AIROLAM,1050.0


---

In [207]:
a_per_share_price = a[['SYMBOL','VALUE_OF_SECURITY_ACQUIRED','NO_OF_SECURITIES_ACQUIRED']]
a_per_share_price = a_per_share_price[a['VALUE_OF_SECURITY_ACQUIRED'] != '-']
a_per_share_price = a_per_share_price.astype({'VALUE_OF_SECURITY_ACQUIRED':'float','NO_OF_SECURITIES_ACQUIRED':'float'})

a_per_share_price

,SYMBOL,VALUE_OF_SECURITY_ACQUIRED,NO_OF_SECURITIES_ACQUIRED
12,DBOL,2039760.0,12000.0
13,SPLPETRO,24069369.0,30000.0
14,SPLPETRO,8455239.0,10633.0
26,SHALBY,2897682.0,20000.0
32,MEP,3922500.0,328000.0
...,...,...,...
4848,MAHICKRA,108600.0,1500.0
4849,MAHICKRA,109950.0,1500.0
4850,DBCORP,11350739.0,126319.0
4851,DBCORP,5221650.0,58003.0


###  Average Price / Share

In [208]:
a_per_share_price['AVGPRICE_PERSHARE'] = a_per_share_price['VALUE_OF_SECURITY_ACQUIRED']/a_per_share_price['NO_OF_SECURITIES_ACQUIRED']

In [209]:
avg_per_share_price = a_per_share_price[['SYMBOL','AVGPRICE_PERSHARE']]
avg_per_share_price = avg_per_share_price.groupby(['SYMBOL']).mean()

In [210]:
# a_per_share_price.groupby('SYMBOL')['PERSHARE_PRICE'].unique()
avg_per_share_price.reset_index(inplace=True)

In [211]:
avg_per_share_price

,SYMBOL,AVGPRICE_PERSHARE
0,ADANIENT,3655.385437
1,ADANIGREEN,2346.793446
2,ADANIPORTS,829.340000
3,ADANITRANS,3565.280000
4,AIROLAM,73.250476
...,...,...
86,VIPCLOTHNG,6.332500
87,VIPIND,584.046503
88,VSTTILLERS,2515.020000
89,WSTCSTPAPR,478.914900


###  Maximun And Minimun Price Per Share

In [212]:
highest_range  = pd.DataFrame(a_per_share_price.groupby('SYMBOL')['AVGPRICE_PERSHARE'].max()).reset_index()
highest_range['HIGH'] = round(highest_range['AVGPRICE_PERSHARE'],2)
highest_range = highest_range[['SYMBOL','HIGH']]
highest_range.head()

,SYMBOL,HIGH
0,ADANIENT,3851.91
1,ADANIGREEN,2424.44
2,ADANIPORTS,829.34
3,ADANITRANS,3565.28
4,AIROLAM,73.25


In [213]:
lowest_range = pd.DataFrame(a_per_share_price.groupby('SYMBOL')['AVGPRICE_PERSHARE'].min()).reset_index()
lowest_range['LOW'] = round(lowest_range['AVGPRICE_PERSHARE'],2)
lowest_range = lowest_range[['SYMBOL','LOW']]
lowest_range.head()

,SYMBOL,LOW
0,ADANIENT,3464.09
1,ADANIGREEN,2223.14
2,ADANIPORTS,829.34
3,ADANITRANS,3565.28
4,AIROLAM,73.25


In [214]:
highest_range.shape == lowest_range.shape == avg_per_share_price.shape

True

---
### Shareholding Percentage Prior And Post 

In [215]:
shareholding_prcnt = a[['SYMBOL','%_SHAREHOLDING_PRIOR','%_SHAREHOLDING_POST']]

In [216]:
shareholding_prcnt = shareholding_prcnt.astype({'%_SHAREHOLDING_PRIOR':'float','%_SHAREHOLDING_POST':'float'})

In [217]:
shareholding_prcnt['BUY%'] = shareholding_prcnt['%_SHAREHOLDING_POST'] - shareholding_prcnt['%_SHAREHOLDING_PRIOR']
shareholding_prcnt

,SYMBOL,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST,BUY%
12,DBOL,16.64,16.66,0.02
13,SPLPETRO,0.42,0.45,0.03
14,SPLPETRO,0.34,0.35,0.01
26,SHALBY,7.18,7.20,0.02
32,MEP,9.19,9.37,0.18
...,...,...,...,...
4848,MAHICKRA,21.80,21.82,0.02
4849,MAHICKRA,21.78,21.80,0.02
4850,DBCORP,4.57,4.63,0.06
4851,DBCORP,4.63,4.67,0.04


In [218]:
prior  = pd.DataFrame(shareholding_prcnt.groupby('SYMBOL')['%_SHAREHOLDING_PRIOR'].max()).reset_index()
post  = pd.DataFrame(shareholding_prcnt.groupby('SYMBOL')['%_SHAREHOLDING_POST'].max()).reset_index()

In [219]:
prior

,SYMBOL,%_SHAREHOLDING_PRIOR
0,ADANIENT,0.32
1,ADANIGREEN,0.22
2,ADANIPORTS,0.00
3,ADANITRANS,0.00
4,AIROLAM,7.68
...,...,...
86,VIPCLOTHNG,0.50
87,VIPIND,1.91
88,VSTTILLERS,22.02
89,WSTCSTPAPR,3.52


---
## Merging Sarts Here.

In [220]:
main_df = avg_per_share_price.merge(highest_range,on='SYMBOL')
main_df = main_df.merge(lowest_range,on='SYMBOL')
main_df = main_df.merge(prior,on='SYMBOL')
main_df = main_df.merge(post,on='SYMBOL')
main_df = main_df.merge(val,on='SYMBOL')
main_df = main_df.merge(t_sec,on='SYMBOL')





In [221]:
main_df['%_BUY'] = main_df['%_SHAREHOLDING_POST'].astype(float) - main_df['%_SHAREHOLDING_PRIOR'].astype(float)

---
### Scrape Current Prices

In [222]:
y = list(main_df['SYMBOL'])

In [223]:

# Python program to illustrate the concept
# of threading
# importing the threading module
import threading
import time
import numpy as np 
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")
import re

start_time = time.time()
y = list(main_df['SYMBOL'])
got1,not_got1,got,not_got,got_mcap,n_got_mcap = [],[],[],[],[],[] 

def promoter_holding(y):
    global got1,not_got1
    
    price1 = {}
    not_happened1 = []
    
    for i in y:
        stockcode = str(i)
        
        stock_url  = f"https://ticker.finology.in/company/{stockcode}"
        response = requests.get(stock_url)
        time.sleep(2)
        soup = BeautifulSoup(response.text, 'lxml')
        if len(re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])) > 0:
            price1[i] = re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])
        else:
            not_happened1.append(i)   
    got1=price1
    not_got1 = not_happened1


    return [price1,not_happened1]

 
    
    
    
    

def realpriceupdate(y):
    global got,not_got
    
    price = {}
    not_happened = []
    
    for i in y:
        stockcode = str(i)
        stock_url  = f"https://www.screener.in/company/{stockcode}/"
        response = requests.get(stock_url)
        time.sleep(2)
        soup = BeautifulSoup(response.text, 'lxml')
        if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
            price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
        else:
            not_happened.append(i)      

    got = price
    not_got = not_happened
            
    return [price,not_happened][1]





def marketcap(y):
    global got_mcap,not_got_mcap
    
    m_cap = {}
    not_happened2 = []
    
    for i in y:
        stockcode = str(i)
        stock_url  = f"https://www.screener.in/company/{stockcode}/"
        response = requests.get(stock_url)
        time.sleep(2)
        soup = BeautifulSoup(response.text, 'lxml')
        d = (soup.find_all('span',{"class":"number"})[0])
        d = d.get_text()
        d = d.replace(',','')
        m_cap[i] = d   
    got_mcap = m_cap
    not_got_mcap = not_happened2

    print()
    return [m_cap,not_happened2]
 
 
if __name__ =="__main__":
    # creating thread
    t1 = threading.Thread(target=promoter_holding, args=(y,))
    t2 = threading.Thread(target=realpriceupdate, args=(y,))
    t3 = threading.Thread(target=marketcap, args=(y,))

    # starting thread 1
    t1.start()
    # starting thread 2
    t2.start()
    
    t3.start()
 
    # wait until thread 1 is completely executed
    a = t1.join()
    # wait until thread 2 is completely executed
    b = t2.join()
    
    c = t3.join()

#     print(a)
#     print(b)

    # print_cube(10)
    # print_square(10)
    print('_'*30,'\n')
    print(got1,'\n')
    print(not_got1,'\n')
    print('_'*30,'\n')
    print(got,'\n')
    print(not_got,'\n')    
    print('_'*30,'\n')
    print(got_mcap,'\n')
    print(not_got_mcap,'\n')
    print(time.time() - start_time)


______________________________ 

{'ADANIENT': ['72.63'], 'ADANIGREEN': ['60.75'], 'ADANIPORTS': ['66.02'], 'ADANITRANS': ['73.87'], 'AIROLAM': ['52.33'], 'ALMONDZ': ['59.8'], 'ANANTRAJ': ['63.03'], 'APCOTEXIND': ['58.22'], 'ARIES': ['52.66'], 'ASAHIINDIA': ['54.24'], 'ASTEC': ['66.77'], 'BECTORFOOD': ['51.14'], 'BSHSL': ['73.27'], 'CHEMBOND': ['67.49'], 'CLEDUCATE': ['52.5'], 'CONFIPET': ['60.08'], 'CONSOFINVT': ['71.53'], 'COSMOFIRST': ['44.24'], 'CREST': ['68.7'], 'DBCORP': ['72.32'], 'DCW': ['44.85'], 'DHAMPURSUG': ['49.08'], 'ELECON': ['59.29'], 'EMMBI': ['58.31'], 'FCL': ['64.97'], 'GANESHBE': ['44.68'], 'GHCL': ['19.03'], 'GNA': ['67.96'], 'GODREJAGRO': ['74.07'], 'GODREJPROP': ['58.48'], 'GREENPLY': ['52.32'], 'GRPLTD': ['41.51'], 'HCLTECH': ['60.72'], 'HERANBA': ['74.87'], 'IPL': ['67.2'], 'IRB': ['34.2'], 'JINDALSTEL': ['61.2'], 'KDDL': ['49.08'], 'KMSUGAR': ['56.52'], 'KPIGREEN': ['54.7'], 'KSOLVES': ['58.94'], 'LAMBODHARA': ['72.95'], 'LINC': ['58.7'], 'MAHICKRA': ['51.33']

---
**Promoter Holding Calculation**

In [224]:
z1 = pd.DataFrame.from_dict([got1])
z1 = z1.T.reset_index()
z1.columns = ['SYMBOL','PROMOTER_HOLDING']
z1['PROMOTER_HOLDING'] = (z1['PROMOTER_HOLDING'].astype(str).str.strip("'[]'")).astype('float')

In [225]:
print(z1.shape)
z1.head()

(90, 2)


,SYMBOL,PROMOTER_HOLDING
0,ADANIENT,72.63
1,ADANIGREEN,60.75
2,ADANIPORTS,66.02
3,ADANITRANS,73.87
4,AIROLAM,52.33


---
**__Current Price Calculation__**

In [226]:
z = pd.DataFrame.from_dict([got])
z = z.T.reset_index()
z.columns = ['SYMBOL','CURRENT_PRICE']
z['CURRENT_PRICE'] = z['CURRENT_PRICE'].str.replace(',','')
z['CURRENT_PRICE'] = z['CURRENT_PRICE'].astype('float')

In [227]:
print(z.shape)
z.head()

(91, 2)


,SYMBOL,CURRENT_PRICE
0,ADANIENT,3833.0
1,ADANIGREEN,2121.0
2,ADANIPORTS,863.0
3,ADANITRANS,3319.0
4,AIROLAM,106.0


---
**Market Cap Calculaton**

In [228]:
z3 = pd.DataFrame.from_dict([got_mcap])
z3 = z3.T.reset_index()
z3.columns = ['SYMBOL','MARKET_CAP']
z3['MARKET_CAP'] = (z3['MARKET_CAP'].astype('float'))

In [229]:
print(z3.shape)
z3.head()

(91, 2)


,SYMBOL,MARKET_CAP
0,ADANIENT,436991.0
1,ADANIGREEN,336013.0
2,ADANIPORTS,186398.0
3,ADANITRANS,370282.0
4,AIROLAM,159.0


In [230]:
main_df = main_df.merge(z,on='SYMBOL')
main_df = main_df.merge(z1,on='SYMBOL')
main_df = main_df.merge(z3,on='SYMBOL')

In [231]:
main_df.head()

,SYMBOL,AVGPRICE_PERSHARE,HIGH,LOW,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST,VALUE_OF_SECURITY_ACQUIRED,NO_OF_SECURITIES_ACQUIRED,%_BUY,CURRENT_PRICE,PROMOTER_HOLDING,MARKET_CAP
0,ADANIENT,3655.385437,3851.91,3464.09,0.32,0.35,1.278702e+10,3486140.0,0.03,3833.0,72.63,436991.0
1,ADANIGREEN,2346.793446,2424.44,2223.14,0.22,0.25,9.351393e+09,4000100.0,0.03,2121.0,60.75,336013.0
2,ADANIPORTS,829.340000,829.34,829.34,0.00,0.00,8.293400e+04,100.0,0.00,863.0,66.02,186398.0
3,ADANITRANS,3565.280000,3565.28,3565.28,0.00,0.00,3.565280e+05,100.0,0.00,3319.0,73.87,370282.0
4,AIROLAM,73.250476,73.25,73.25,7.68,0.01,7.691300e+04,1050.0,-7.67,106.0,52.33,159.0


### ***DATA   CLEANED !!!***
---

In [232]:
index = np.where(main_df.VALUE_OF_SECURITY_ACQUIRED >10000000)
data = main_df.iloc[index]
index1 = np.where(data.PROMOTER_HOLDING > 40)
data = data.iloc[index1]
index3 = np.where(data.MARKET_CAP > 1000)
data = data.iloc[index3]

In [233]:
print(data.shape)
data.head()

(36, 12)


,SYMBOL,AVGPRICE_PERSHARE,HIGH,LOW,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST,VALUE_OF_SECURITY_ACQUIRED,NO_OF_SECURITIES_ACQUIRED,%_BUY,CURRENT_PRICE,PROMOTER_HOLDING,MARKET_CAP
0,ADANIENT,3655.385437,3851.91,3464.09,0.32,0.35,1.278702e+10,3486140.0,0.03,3833.0,72.63,436991.0
1,ADANIGREEN,2346.793446,2424.44,2223.14,0.22,0.25,9.351393e+09,4000100.0,0.03,2121.0,60.75,336013.0
6,ANANTRAJ,85.835817,88.98,82.61,0.01,0.02,1.411685e+07,165000.0,0.01,115.0,63.03,3722.0
9,ASAHIINDIA,561.859993,561.86,561.86,5.64,5.67,4.077306e+07,72568.0,0.03,618.0,54.24,15011.0
10,ASTEC,1896.964998,1899.47,1894.46,64.54,64.80,1.897650e+08,100036.0,0.26,2174.0,66.77,4262.0


---

In [234]:
print(main_df.shape)
main_df.head()

(90, 12)


,SYMBOL,AVGPRICE_PERSHARE,HIGH,LOW,%_SHAREHOLDING_PRIOR,%_SHAREHOLDING_POST,VALUE_OF_SECURITY_ACQUIRED,NO_OF_SECURITIES_ACQUIRED,%_BUY,CURRENT_PRICE,PROMOTER_HOLDING,MARKET_CAP
0,ADANIENT,3655.385437,3851.91,3464.09,0.32,0.35,1.278702e+10,3486140.0,0.03,3833.0,72.63,436991.0
1,ADANIGREEN,2346.793446,2424.44,2223.14,0.22,0.25,9.351393e+09,4000100.0,0.03,2121.0,60.75,336013.0
2,ADANIPORTS,829.340000,829.34,829.34,0.00,0.00,8.293400e+04,100.0,0.00,863.0,66.02,186398.0
3,ADANITRANS,3565.280000,3565.28,3565.28,0.00,0.00,3.565280e+05,100.0,0.00,3319.0,73.87,370282.0
4,AIROLAM,73.250476,73.25,73.25,7.68,0.01,7.691300e+04,1050.0,-7.67,106.0,52.33,159.0


In [235]:
path = r'C:\Users\user\Desktop\Stocks_Automated\Cleaned_Data.csv'
path1 = r'C:\Users\user\Desktop\Stocks_Automated\Full_Data.csv'
# path2 = r'C:\Users\user\Desktop\Stocks_Automated\PRMTRHLDN_ABV40_ALL_18OCT.csv'
# path3 = r'C:\Users\user\Desktop\Stocks_Automated\ABOVE2CR_18OCT.csv'

In [250]:
data.to_csv(path,index=False)
main_df.to_csv(path1,index=False)
# prmtrhldng_abv40_all.to_csv(path2)

# ENDS

---
### ROUGH BELOW...

In [237]:
# def marketcap():
    

In [238]:
# def promoter_holding(y):
#     global got1,not_got1
    
#     price1 = {}
#     not_happened1 = []
    
#     for i in y:
#         stockcode = str(i)
        
#         stock_url  = f"https://ticker.finology.in/company/{stockcode}"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])) > 0:
#             price1[i] = re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])
#         else:
#             not_happened1.append(i)   
#     got1=price1
#     not_got1 = not_happened1


#     return [price1,not_happened1]

In [239]:
# def promoter_holding(y):
#     global got1,not_got1
    
#     m_cap = {}
#     not_happened2 = []
    
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('span',{"class":"number"})) > 0:
#             m_cap[i] = (soup.find_all('span',{"class":"number"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)   
#     got_mcap=m_cap
#     not_got2 = not_happened2

#     print()
#     return [price1,not_happened2]

In [240]:
# abv_1cr = main_df[main_df['VALUE_OF_SECURITY_ACQUIRED'] > 10000000]

In [241]:
# less_thn_prmtr_price = abv_1cr[abv_1cr['AVGPRICE_PERSHARE'] > abv_1cr['CURRENT_PRICE']]

In [242]:
# prmtrhldng_abv40 = less_thn_prmtr_price[less_thn_prmtr_price['PROMOTER_HOLDING'] > 40]
# prmtrhldng_abv40_all = abv_10cr[abv_10cr['PROMOTER_HOLDING'] > 40]

# prmtrhldng_abv40.head()

In [243]:
# def marketcap(y):
#     global got_mcap,not_got_mcap
    
#     m_cap = {}
#     not_happened2 = []
    
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         d = (soup.find_all('span',{"class":"number"})[0])
#         d = d.get_text()
#         d = d.replace(',','')
#         m_cap[i] = d   
#     got_mcap = m_cap
#     not_got_mcap = not_happened2

#     print()
#     return [m_cap,not_happened2]

In [244]:
# z3 = pd.DataFrame.from_dict([got_mcap])
# z3 = z3.T.reset_index()
# z3.columns = ['SYMBOL','MARKET_CAP']
# z3['MARKET_CAP'] = (z3['MARKET_CAP'].astype('float'))
# z3

In [245]:
# m_cap = {}
# not_happened2 = []
# for i in y:
#     stockcode = str(i)
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     time.sleep(2)
#     soup = BeautifulSoup(response.text, 'lxml')
#     d = (soup.find_all('span',{"class":"number"})[0])
#     d = d.get_text()
#     d = d.replace(',','')
#     m_cap[i] = d

# print(m_cap)


In [246]:
# stockcode = str('ADANIENT')

# stock_url  = f"https://www.screener.in/company/{stockcode}/"
# response = requests.get(stock_url)
# time.sleep(2)
# soup = BeautifulSoup(response.text, 'lxml')
# d = (soup.find_all('span',{"class":"number"})[0])
# d = d.get_text()
# d = d.replace(',','')
# print(d)


In [247]:
# def realpriceupdate(y):
#     global got,not_got
    
#     price = {}
#     not_happened = []
    
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
#             price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)      

#     got = price
#     not_got = not_happened
            
#     return [price,not_happened][1]

In [248]:
#
# # Python program to illustrate the concept
# # of threading
# # importing the threading module
# import threading
# import time
# import numpy as np 
# import pandas as pd 
# import requests
# from bs4 import BeautifulSoup
# import warnings
# warnings.filterwarnings("ignore")
# import re
 
# y = ['ADANIENT',
#  'ADANIGREEN',
#  'ADANIPORTS']
# f1,f2,f3,f4 = [],[],[],[] 

# def promoter_holding(y):
#     global f1,f2
    
#     price1 = {}
#     not_happened1 = []
    
#     for i in y:
#         stockcode = str(i)
        
#         stock_url  = f"https://ticker.finology.in/company/{stockcode}"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])) > 0:
#             price1[i] = re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])
#         else:
#             not_happened1.append(i)   
#     f1=price1
#     f2 = not_happened1


#     return [price1,not_happened1]

 

# def realpriceupdate(y):
#     global f3,f4
    
#     price = {}
#     not_happened = []
    
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
#             price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)      

#     f3 = price
#     f4 = not_happened
            
#     return [price,not_happened][1]
 
 
# if __name__ =="__main__":
#     # creating thread
#     t1 = threading.Thread(target=promoter_holding, args=(y,))
#     t2 = threading.Thread(target=realpriceupdate, args=(y,))

#     # starting thread 1
#     t1.start()
#     # starting thread 2
#     t2.start()
 
#     # wait until thread 1 is completely executed
#     a = t1.join()
#     # wait until thread 2 is completely executed
#     b = t2.join()

# #     print(a)
# #     print(b)

#     # print_cube(10)
#     # print_square(10)
#     print('_'*30)
#     print(f1,'\n')
#     print(f2,'\n')
#     print('_'*30)
#     print(f3,'\n')
#     print(f4,'\n')

In [249]:
******PLEASE READ  BEFORE ACCEPTING*******      --->         FACING  ISSUE  IN  CALLING  MULTIPLE   FUNCTION AT A SAME TIME USING THREADING

SyntaxError: invalid syntax (<ipython-input-249-3a2ff5745d61>, line 1)

In [ ]:
# for i in z.CURRENT_PRICE:
#     if ',' in i:
#         z['new']= int(i.replace(',', ''))
#     else:
#         z['new']= float(i)
        
    
# z.head()

In [ ]:
# for i in (z.CURRENT_PRICE):
#     print(i)
# z
    

## Rough Start

In [ ]:
# import time 

# def promoter_holding(y):
#     price1 = {}
#     not_happened1 = []
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://ticker.finology.in/company/{stockcode}"
#         response = requests.get(stock_url)
#         time.sleep(5)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])) > 0:
#             price1[i] = re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])
#         else:
#             not_happened1.append(i)

            
            
#     return(price1,not_happened1)

In [ ]:
# import time 

# def realpriceupdate(y):
#     price = {}
#     not_happened = []
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         time.sleep(3)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
#             price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)

            
            
#     return(price,not_happened)

In [ ]:
# got,not_got = realpriceupdate(y)
# got,not_got = t2.start(y)


In [ ]:
# # Python program to illustrate the concept
# # of threading
# # importing the threading module
# import threading
# import time

# def promoter_holding(y):
#     price1 = {}
#     print(y)
#     not_happened1 = []
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://ticker.finology.in/company/{stockcode}"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])) > 0:
#             price1[i] = re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])
#         else:
#             not_happened1.append(i)   
            
#     return(price1,not_happened1)



# def realpriceupdate(y):
#     price = {}
    
#     not_happened = []
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         time.sleep(2)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
#             price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)      
            
#     return(price,not_happened)



# if __name__ == '__main__':

# # creating thread
#     t1 = threading.Thread(target=promoter_holding, args=(y,))
#     t2 = threading.Thread(target=realpriceupdate, args=(y,))
#     t1.start()
#     t2.start()
    
# # starting thread 1
# #     got1,not_got1 = t1.start()

    
    
    
# # t1.start()

# # starting thread 2


# #     got,not_got = t2.start()

    
    

# # t2.start()

# # wait until thread 1 is completely executed
#     print(t1.join())
# # wait until thread 2 is completely executed
#     print(t2.join())

# #     print(t1.value)
# # both threads completely executed
# print("Done!")

In [ ]:
#
# # Python program to illustrate the concept
# # of threading
# # importing the threading module
# import threading
# import time 
 
 
# def print_cube(num):
#     # function to print cube of given num
#     c = 0 
#     # function to print square of given num
#     time.sleep(2)
#     print("Cube: {}" .format(num * num * num))
#     print(time.time())
 

# def print_square(num):
#     c = 0 
#     # function to print square of given num
#     for i in range(1,50000):
#         c +=1
#     print("Square: {}" .format(num * num))
#     print(time.time())

 
 
# if __name__ =="__main__":
#     # creating thread
#     t1 = threading.Thread(target=print_square, args=(10,))
#     t2 = threading.Thread(target=print_cube, args=(10,))
 
#     # starting thread 1
#     t1.start()
#     # starting thread 2
#     t2.start()
 
#     # wait until thread 1 is completely executed
# #     t1.join()
#     # wait until thread 2 is completely executed
# #     t2.join()
 
#     # both threads completely executed
#     print("Done!")

In [ ]:
# # Python program to illustrate the concept
# # of threading
# # importing the threading module
# import threading
# import time
# def print_cube(num):
#     time.sleep(5)
#     # function to print cube of given num
#     print("Cube: {}" .format(num * num * num),'\n')
    

    
# def print_square(num):
#     time.sleep(5)
#     # function to print square of given num
#     print("Square: {}" .format(num * num),'\n')
    

# # creating thread
# # t1 = threading.Thread(target=print_square, args=(10,))
# # t2 = threading.Thread(target=print_cube, args=(10,))
# # starting thread 1
# # t1.start()
# # starting thread 2
# # t2.start()
# # wait until thread 1 is completely executed
# #t1.join()
# # wait until thread 2 is completely executed
# #t2.join()
# # both threads completely executed
# print("Done!")

In [ ]:
# got1,not_got1 = promoter_holding(y)
# got1,not_got1 = t1.start(y)
                 
# if __name__ == '__main__':
    
#     got1,not_got1 = t1.start(y)
# y

In [ ]:
# stockcode = str('UPL')
# stock_url  = f"https://www.screener.in/company/{stockcode}/"
# response = requests.get(stock_url)
# response
# soup = BeautifulSoup(response.text, 'lxml')
# soup
# #soup.find_all('button ',{"class":"button-plain"})
# soup.find_all('td')
# # soup.find_all('button',{"class":"button-plain"})



In [ ]:
# nowrap value
# number

# class="text"
# class="button-plain"

In [ ]:
# https://stockedge.com/Stocks/upl/7381
#     https://ticker.finology.in/company/UPL

In [ ]:
# stockcode = str('UPL')
# stock_url  = f"https://stockedge.com/Stocks/{stockcode}/7381"
# response = requests.get(stock_url)
# response
# soup = BeautifulSoup(response.text, 'lxml')
# soup
# #soup.find_all('button ',{"class":"button-plain"})
# soup.find_all('td',{"data-name":"shp-data"})[
# # soup.find_all('button',{"class":"button-plain"})


In [ ]:
# import re
# stockcode = str('ITC')
# stock_url  = f"https://ticker.finology.in/company/{stockcode}"
# response = requests.get(stock_url)
# response
# soup = BeautifulSoup(response.text, 'lxml')
# soup
# re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])

In [ ]:
# y

In [ ]:
# import re
# stockcode = str('ITC')
# stock_url  = f"https://ticker.finology.in/company/{stockcode}"
# response = requests.get(stock_url)
# response
# soup = BeautifulSoup(response.text, 'lxml')
# soup
# re.findall(r'\d+.\d*', soup.find_all('p')[15].text[1:])

## Rough End

In [ ]:
# z['CURRENT_PRICE'] = int(z['CURRENT_PRICE'].replace(',', ''))
# # int(rectify.replace(',', ''))

In [ ]:
# (price)

In [ ]:
# prices = []
# for i in y:
# #     print(realpriceupdate(i))
#     prices.append(realpriceupdate(i))
    

In [ ]:
# print(len(prices))
# len(y)
# print(realpriceupdate(y[23]))
# prices
# # res = dict(zip(y,prices))
# # res



# Pending...

In [ ]:
# z['CURRENT_PRICE'] = z['CURRENT_PRICE'].str.strip(",")

In [ ]:
# for i in z['CURRENT_PRICE']:
#     if ',' in i:
#         i.replace(',','')
#         print(i)

In [ ]:
# abv_10cr['CURRENT_PRICE'].astype(float) < abv_10cr['AVGPRICE_PERSHARE']

In [ ]:
# pending

# Pending...

In [ ]:
# main_df = main_df.merge(z,on='SYMBOL')


In [ ]:
# def realpriceupdate(i):
#     stockcode = str(i)
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     price = []
#     if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
        
# #         price = soup.find_all('div',{"class":"flex flex-align-center"})[0]
#         price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]
#     else:
#         realpriceupdate(stockcode)
    


#     return(price)

In [ ]:
#

# def realpriceupdate(y):
#     price = {}
#     not_happened = []
#     for i in y:
#         stockcode = str(i)
#         stock_url  = f"https://www.screener.in/company/{stockcode}/"
#         response = requests.get(stock_url)
#         soup = BeautifulSoup(response.text, 'lxml')
#         if len(soup.find_all('div',{"class":"flex flex-align-center"})) > 0:
#             price[i] = (soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
# #             price.append(soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:])
#         else:
#             not_happened.append(i)

            
            
#     return(price,not_happened)


In [ ]:
# a,b = realpriceupdate(y)

In [ ]:
# a1,b1 = realpriceupdate(b)

In [ ]:
# price = {**a,**a1}


In [ ]:
# price

In [ ]:
# z.info()

In [ ]:
# import time 

# def realpriceupdate(i):
#     stockcode = str(i)
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]
#     return(price)


In [ ]:
# stockcode = "DHAMPURSUG"
# print(stockcode)

In [ ]:
# # stock_url  = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol='+str(stockcode)
# stock_url  = f"https://www.screener.in/company/{stockcode}/"

# print(stock_url)

# response = requests.get(stock_url)


In [ ]:
# print(response)

In [ ]:
# soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
# price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]

In [ ]:
# print(price)

In [ ]:
# def real_price():
#     stockcode = "DHAMPURSUG"
#     stock_url  = f"https://www.screener.in/company/{stockcode}/"
#     response = requests.get(stock_url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     price = soup.find_all('div',{"class":"flex flex-align-center"})[0].find('span').text[1:]
#     print(price)

    